<a href="https://colab.research.google.com/github/msun-dev/deadlock_analysis/blob/main/ranked__games_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Show me the truth!

## Libraries import

In [2]:
import requests  # alma mater
import json      # JSON
import time      # Delaying requests
import random    # Who doesnt like RNG?

## Variables

### User-defined variables

In [15]:
account_id = 863727003 # @param {type:"number", placeholder:"Your Steam 3 ID. Use service like rep.tf."}
request_delay_sec = 6.0 # @param {type:"number", placeholder:"Use 6.0 without API key"}

### Constants

In [14]:
ranks = ["obscurus", "initiate", "seeker", "alchemist", "arcanist", "ritualist", "emissary", "archon", "oracle", "phantom", "ascendant", "eternus"]
#            0           1          2           3           4           5            6          7         8          9          10           11

match_history_endpoint = f'https://data.deadlock-api.com/v2/players/*/match-history' #
match_stats_endpoint = f'https://data.deadlock-api.com/v1/matches/*/metadata'        # 10req/min & 100req/h. 100req/s with API key
rank_badges_endpoint = 'https://assets.deadlock-api.com/v2/ranks?language=english'   #

games_data = {
    # rank_id: [wins, losses]
    8: [0, 0],
    9: [0, 0],
    10: [0, 0],
    11: [0, 0]
}

## Functions

### Request functions

In [5]:
def convert_request_to_obj(data: str) -> dict:
  obj = json.loads(data)
  return obj

def get_api_request(api_endpoint: str) -> dict:
  req = requests.get(api_endpoint).text
  obj = convert_request_to_obj(req)
  return obj

def get_player_matches_raw(player_id: int) -> dict:
  player_matches = get_api_request(match_history_endpoint.replace("*", str(player_id)))
  return player_matches

def get_match_data(match_id: int) -> dict:
  match_data = get_api_request(match_stats_endpoint.replace("*", str(match_id)))
  match_data = match_data['match_info']
  return match_data

### Functions for data extraction

In [6]:
def get_player_match_ids(player_matches: dict) -> list: # TODO: Add array return type
  matches = [i['match_id'] for i in player_matches['matches']]
  return matches

def get_matches_count(match_ids) -> int:
  match_count = len(match_ids)
  return match_count

def get_random_match_id(player_matches) -> int:
  match_id = random.choice(player_matches)
  return match_id

def get_team_rank_ids(match_data: dict) -> list:
  team0 = match_data['average_badge_team0']
  team1 = match_data['average_badge_team1']
  return [team0, team1]

def get_average_lobby_rank(team_ranks: list) -> int:
  # TODO: Implement!
  return 0

def get_players_team(match_data: dict) -> bool:
  for player in match_data['players']:
    if player['account_id'] == account_id:
      players_team = player['team'][-1]
      return players_team

def get_team_badge(match_data: dict, team_id: bool) -> int:
  if team_id == 0:
    return match_data['average_badge_team0']
  else:
    return match_data['average_badge_team1']

def get_team_outcome(match_data: dict, team_id: bool) -> bool:
  if team_id != match_data['winning_team'][-1]:
    return 0
  else:
    return 1

def get_team_stats(match_data, team_id: bool) -> list:
  team_badge = get_team_badge(match_data, team_id)
  team_outcome = get_team_outcome(match_data, team_id)
  return [team_badge, team_outcome]

def get_rank_id(rank_id: int) -> int:
  return int(str(rank_id)[:-1])

def get_subrank(rank_id: int) -> int:
  return int(str(rank_id)[-1])

def get_rank_string(rank_id: int) -> str:
  rank_name = get_rank_id(rank_id)
  rank_number = get_subrank(rank_id)
  return f'{ranks[rank_name].title()} {rank_number}'

### Main functions

In [7]:
# Main functions

def main() -> None:
  player_match_ids = get_player_match_ids(get_player_matches_raw(account_id))
  for i, match_id in enumerate(player_match_ids):
    print(f'Parsing match #{i} with ID {match_id}')
    match_data = get_match_data(match_id)
    players_team_stats = get_team_stats(match_data, get_players_team(match_data))
    add_game(players_team_stats)
    print(f'Added game with stats: {get_rank_string(players_team_stats[0])} - {"Lost" if players_team_stats[1] == 0 else "Win"}')
    time.sleep(request_delay_sec)
  print(analyze_output())

def add_game(team_stats: list) -> None:
  team_rank_id = get_rank_id(team_stats[0])
  team_outcome = team_stats[1]
  if team_rank_id in games_data.keys():
    rank_stats = games_data[team_rank_id]
    if team_outcome == 1:
      rank_stats[0] += 1
    else:
      rank_stats[1] += 1
  else:
    print(f'There was a game with rank that is not exist in the dict: {team_rank_id} - {get_rank_string(team_stats[0])}')

def analyze_output() -> str:
  output = ''
  for key in games_data.keys():
    games_total = games_data[key][0] + games_data[key][1]
    rank_string = ranks[key].title()
    rank_games = f'{games_data[key][0]} W - {games_data[key][1]} L. {games_total} games total'
    rank_winrate = f'WR - {games_total / 100 * games_data[key][0]}%'
    rank_string = f'{rank_string} - {rank_games} - {rank_winrate}. '
    output += rank_string
  return output

In [8]:
main() # LET THE TRUTH UNFOLD

Parsing match #0 with ID 32766477
Added game with stats: Phantom 6 - Win
Parsing match #1 with ID 32765361
Added game with stats: Phantom 1 - Lost
Parsing match #2 with ID 32764391
Added game with stats: Phantom 2 - Lost
Parsing match #3 with ID 32761187
Added game with stats: Oracle 5 - Win
Parsing match #4 with ID 32760012
Added game with stats: Phantom 5 - Lost
Parsing match #5 with ID 32759644
Added game with stats: Phantom 4 - Win
Parsing match #6 with ID 32756190
Added game with stats: Ascendant 2 - Win
Parsing match #7 with ID 32755463
Added game with stats: Oracle 2 - Lost
Parsing match #8 with ID 32754692
Added game with stats: Oracle 3 - Win
Parsing match #9 with ID 32753966
Added game with stats: Oracle 3 - Win
Parsing match #10 with ID 32722563
Added game with stats: Ascendant 4 - Win
Parsing match #11 with ID 32721456
Added game with stats: Oracle 3 - Lost
Parsing match #12 with ID 32720060
Added game with stats: Oracle 2 - Win
Parsing match #13 with ID 32717743
Added game

...
At this point (when I run main() for the first time) I remembered that Valve (or deadlockapi, rather) doesnt have real ranks data. :| Need to click through every game now. Would be pretty slow.

Also you can't use `f'Line\nNew line'` in python 3.11 which is used by colab  

##Misc code:

### Pretty output

In [ ]:
# For test purposes
player_matches = get_player_matches_raw(account_id)
player_match_ids = get_player_match_ids(player_matches)
player_match_count = get_matches_count(player_match_ids)
random_match_id = get_random_match_id(player_match_ids)
random_match_data = get_match_data(random_match_id)
#teams_ranks = get_lobby_team_rank_ids(random_match_data)
players_team = get_players_team(random_match_data)
players_team_stats = get_team_stats(random_match_data, players_team)
print(f"""
Player with ID {account_id} has {player_match_count} matches available.
Picked random match ID: {random_match_id}.
Player is in team: {players_team}.
Players team stats: {get_rank_string(players_team_stats[0])} - {"Lost" if players_team_stats[1] == 0 else "Win"}
""")

### Get keys from match data

In [ ]:
random_match_data.keys()

In [ ]:
random_match_data['winning_team']

###Badges definition

In [12]:
r = get_api_request(rank_badges_endpoint)
r

[{'tier': 0,
  'name': 'Obscurus',
  'images': {'large': 'https://assets.deadlock-api.com/images/ranks/rank0/badge_lg.png',
   'large_webp': 'https://assets.deadlock-api.com/images/ranks/rank0/badge_lg.webp',
   'small': 'https://assets.deadlock-api.com/images/ranks/rank0/badge_sm_psd.png',
   'small_webp': 'https://assets.deadlock-api.com/images/ranks/rank0/badge_sm.webp'},
  'color': '#333333'},
 {'tier': 1,
  'name': 'Initiate',
  'images': {'large': 'https://assets.deadlock-api.com/images/ranks/rank1/badge_lg.png',
   'large_webp': 'https://assets.deadlock-api.com/images/ranks/rank1/badge_lg.webp',
   'large_subrank1': 'https://assets.deadlock-api.com/images/ranks/rank1/badge_lg_subrank1.png',
   'large_subrank1_webp': 'https://assets.deadlock-api.com/images/ranks/rank1/badge_lg_subrank1.webp',
   'large_subrank2': 'https://assets.deadlock-api.com/images/ranks/rank1/badge_lg_subrank2.png',
   'large_subrank2_webp': 'https://assets.deadlock-api.com/images/ranks/rank1/badge_lg_subran